## Simulation
#### JGWJ | 06/29/2021

This is a simple simulation problem: sample from every batch produced has to go through a QC inspection, testing and a final QA inspection. We are to limit total testing time to within 30 minutes and using the least amount of manpower/equipment in the lab.

Running simulation for 6 items/hour, we should post 2 QC and 4 testing equipments (testers) and 1 QA to keep the total process time at 29.51 minutes and average wait time between steps at 2.10 minutes, which is the least combination of resources. 


##### Work steps
1. Setup constants and global variable
2. Setup Inspection Class
3. Setup Item Process
4. Setup Item Arrival
5. Setup & Run Simulation for arrival 6 items/hour

We will be using simpy to run simulation of a production line with following attributes:
-	item arrival according to a Poisson distribution with λ1 = 6 per hour (i.e., mean interarrival rate = 10 minutes), 
-	QC with exponential service time with mean rate = 5 minutes, 
-	Testing with exponential service time with mean rate = 20 minutes, 
-	Allocating to an available QA with shortest queue 
-	QA inspection with time uniformly distributed between 2.5 minutes and 5 minutes, and
-	running the simulation within 8 hours.


In [114]:
# !pip3 install simpy
import simpy
import random
import pandas as pd
print("libraries imported")

libraries imported


##### 1. Setup constants and global variable

In [115]:
random_seed = 54
num_qc = 4          # Number of QC staff
num_testmac = 2     # Number of testing machines
num_qa = 1          # Number of QA staff
qc_time = 5         # Minutes it takes to clear QC check
testmac_time = 20   # Minutes it takes to clear tests
qa_time_min = 2.5   # Minimum Minutes it takes to go through QA check
qa_time_max = 5     # Maximum Minutes it takes to go through QA check
arrival = 0.1       # 0.1 item per minute 
sim_time = 8*60     # Simulation time in minutes

time_df = []        # dataframe for keeping times 
total_items = 0     # total items checked count

print("Constants and global variables declared")

Constants and global variables declared


##### 2. Setup Inspection Class
An item quality inspection process has a limited number of QC, testing equipment, and QA officers.  

In [116]:
#define Security class, this is the simulation environment
class Inspection(object):
    
    def __init__(self, env, num_qc, num_testmac, num_qa, qc_time, testmac_time, qa_time_min, qa_time_max):
        
        #define simulation parameters
        #save arguments as member variables
        self.env = env
        self.num_qc = num_qc
        self.num_testmac = num_testmac 
        self.num_qa = num_qa
        self.qc_time = qc_time
        self.testmac_time = testmac_time
        self.qa_time_min = qa_time_min 
        self.qa_time_max = qa_time_max
        
        #declare resources
        self.QC = simpy.Resource(env, num_qc)
        self.TESTMAC = simpy.Resource(env, num_testmac)
        self.QA = []
        
        for i in range(num_qa):
            self.QA.append(simpy.Resource(env,1))
    
    #QC inspection process
    def qccheck(self, item):
        yield self.env.timeout(random.expovariate(1/qc_time)) #exponential distributed
    
    #Machine testing process
    def testmac(self, item):
        yield self.env.timeout(random.expovariate(1/testmac_time)) #exponential distributed
    
    #Personal scanner process
    def qacheck(self, item):
        yield self.env.timeout(random.uniform(qa_time_min, qa_time_max)) #uniformly distributed 

print("Inspection class setup complete")

Inspection class setup complete


##### 3. Setup Item  Process
The item produced arrives at lab for quality inspection:first QC check, then testing, followed by QA check, waits for the process to finish to clear inspection.

In [117]:
def Items(env, name, inspection):
    
    global total_items
    global time_df
    
    inspectionStartTime = env.now
    
    #Go through QC check first
    with inspection.QC.request() as request:
        
        yield request #wait for available agent
        qcCheckStartTime = env.now
        
        yield env.process(inspection.qccheck(name)) #get QC inspection
        
        qcCheckTime = env.now - qcCheckStartTime
    
    #Go through testing second
    with inspection.TESTMAC.request() as request:
        
        yield request #wait for available agent
        testCheckStartTime = env.now
        
        yield env.process(inspection.testmac(name)) #go through testing
        
        testCheckTime = env.now - testCheckStartTime
    
    #after testing, see which QA's load is lightest
    shortQueue = 0
                
    for i in range(1, inspection.num_qa):
        if len((inspection.QA)[i].queue) < len(inspection.QA[shortQueue].queue):
            shortQueue = i
                
    with inspection.QA[shortQueue].request() as request: 
        
        yield request #join shortest scanner queue, wait for turn
                
        qaStartTime = env.now
        
        yield env.process(inspection.qacheck(name)) #go through personal scanner
        
        #item clears quality inspection
        #print('Item %s cleared quality inspection' %name) 
        
        total_items = total_items + 1
        
        #saving times into dataframe
        qaCheckTime = env.now - qaStartTime
        
        totalTime = env.now - inspectionStartTime 
        
        time_df.append({'name':name,
                       'QC_Check_Time': qcCheckTime,
                        'Testing_Machine_Time': testCheckTime,
                       'QA_Check_Time': qaCheckTime,
                       'Total_Time': totalTime}) 

print("Items setup complete")

Items setup complete


##### 4. Setup Item_Arrival 
Create a item check and keep creating items approx. every 1/arrival minutes.

In [118]:
def Item_Arrival(env, arrival, num_qc, num_testmac, num_qa, qc_time, testmac_time, qa_time_min, qa_time_max):
    
    i = 0 #item id
    
    # Create the item check
    itemcheck = Inspection(env, num_qc, num_testmac, num_qa, qc_time, testmac_time, qa_time_min, qa_time_max)

    # Create more passengers while the simulation is running
    while True:
        yield env.timeout(random.expovariate(arrival))
        i += 1
        
        env.process(Items(env, 'Item %d' % i, itemcheck))
        
print("Item arrival setup complete")

Item arrival setup complete


##### 5. Setup & Run Simulation for arrival 0.1/minute
Here we will pass the constants to start the process simulation. 

- QC 2 | Test Equipment 4 | QA 1 | total time (minutes) 29.400401 | avg wait time (minutes) 1.317073

Initial result shows we should pick 2 QC, 4 testing equipments, and 1 QA with total time in process 29.400401 minutes and average wait time of 1.317073 minutes. Since each run has a different result, I'll run it for 20x later to determine an average of average wait time.

In [119]:
sim = 0

for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
        
            random.seed(54)  # This helps reproducing the results

            df=[]
            time_df = []

            #print('Airport Security Check %s' %sim)

            #create an environment and start the passenger arrival process
            env = simpy.Environment()

            #0.1 arrival, i QC, j testing machine, k QA 
            env.process(Item_Arrival(env, arrival, i, j, k, qc_time, testmac_time, qa_time_min, qa_time_max)) 

            #execute
            env.run(until= sim_time)

            print("Simulation %s Complete" %sim)
            sim += 1

            #save times
            df = pd.DataFrame(time_df)
            df['Wait_time'] = round(df['Total_Time'] - df['QC_Check_Time'] - df['Testing_Machine_Time'] - df['QA_Check_Time'])

            print(i, j, k, df.describe().loc['mean'])
         

Simulation 0 Complete
1 1 1 QC_Check_Time             6.060686
Testing_Machine_Time     44.342765
QA_Check_Time             3.395028
Total_Time              274.852307
Wait_time               221.100000
Name: mean, dtype: float64
Simulation 1 Complete
1 1 2 QC_Check_Time             6.060686
Testing_Machine_Time     44.342765
QA_Check_Time             3.395028
Total_Time              274.852307
Wait_time               221.100000
Name: mean, dtype: float64
Simulation 2 Complete
1 1 3 QC_Check_Time             6.060686
Testing_Machine_Time     44.342765
QA_Check_Time             3.395028
Total_Time              274.852307
Wait_time               221.100000
Name: mean, dtype: float64
Simulation 3 Complete
1 1 4 QC_Check_Time             6.060686
Testing_Machine_Time     44.342765
QA_Check_Time             3.395028
Total_Time              274.852307
Wait_time               221.100000
Name: mean, dtype: float64
Simulation 4 Complete
1 1 5 QC_Check_Time             6.060686
Testing_Machine_T

Name: mean, dtype: float64
Simulation 41 Complete
1 5 6 QC_Check_Time            5.039239
Testing_Machine_Time    18.825073
QA_Check_Time            3.673433
Total_Time              37.954067
Wait_time               10.395833
Name: mean, dtype: float64
Simulation 42 Complete
1 5 7 QC_Check_Time            5.039239
Testing_Machine_Time    18.825073
QA_Check_Time            3.673433
Total_Time              37.954067
Wait_time               10.395833
Name: mean, dtype: float64
Simulation 43 Complete
1 5 8 QC_Check_Time            5.039239
Testing_Machine_Time    18.825073
QA_Check_Time            3.673433
Total_Time              37.954067
Wait_time               10.395833
Name: mean, dtype: float64
Simulation 44 Complete
1 5 9 QC_Check_Time            5.039239
Testing_Machine_Time    18.825073
QA_Check_Time            3.673433
Total_Time              37.954067
Wait_time               10.395833
Name: mean, dtype: float64
Simulation 45 Complete
1 6 1 QC_Check_Time            4.448118
Testin

Simulation 83 Complete
2 1 3 QC_Check_Time             7.057690
Testing_Machine_Time     22.170537
QA_Check_Time             3.755197
Total_Time              259.303187
Wait_time               226.250000
Name: mean, dtype: float64
Simulation 84 Complete
2 1 4 QC_Check_Time             7.057690
Testing_Machine_Time     22.170537
QA_Check_Time             3.755197
Total_Time              259.303187
Wait_time               226.250000
Name: mean, dtype: float64
Simulation 85 Complete
2 1 5 QC_Check_Time             7.057690
Testing_Machine_Time     22.170537
QA_Check_Time             3.755197
Total_Time              259.303187
Wait_time               226.250000
Name: mean, dtype: float64
Simulation 86 Complete
2 1 6 QC_Check_Time             7.057690
Testing_Machine_Time     22.170537
QA_Check_Time             3.755197
Total_Time              259.303187
Wait_time               226.250000
Name: mean, dtype: float64
Simulation 87 Complete
2 1 7 QC_Check_Time             7.057690
Testing_Mach

2 5 4 QC_Check_Time            5.939611
Testing_Machine_Time    18.668013
QA_Check_Time            3.657410
Total_Time              29.087774
Wait_time                0.820000
Name: mean, dtype: float64
Simulation 121 Complete
2 5 5 QC_Check_Time            5.939611
Testing_Machine_Time    18.668013
QA_Check_Time            3.657410
Total_Time              29.087774
Wait_time                0.820000
Name: mean, dtype: float64
Simulation 122 Complete
2 5 6 QC_Check_Time            5.939611
Testing_Machine_Time    18.668013
QA_Check_Time            3.657410
Total_Time              29.087774
Wait_time                0.820000
Name: mean, dtype: float64
Simulation 123 Complete
2 5 7 QC_Check_Time            5.939611
Testing_Machine_Time    18.668013
QA_Check_Time            3.657410
Total_Time              29.087774
Wait_time                0.820000
Name: mean, dtype: float64
Simulation 124 Complete
2 5 8 QC_Check_Time            5.939611
Testing_Machine_Time    18.668013
QA_Check_Time     

2 9 9 QC_Check_Time            5.939611
Testing_Machine_Time    18.593960
QA_Check_Time            3.663585
Total_Time              29.018892
Wait_time                0.840000
Name: mean, dtype: float64
Simulation 162 Complete
3 1 1 QC_Check_Time             7.025047
Testing_Machine_Time     27.741707
QA_Check_Time             3.598761
Total_Time              257.936179
Wait_time               219.625000
Name: mean, dtype: float64
Simulation 163 Complete
3 1 2 QC_Check_Time             7.025047
Testing_Machine_Time     27.741707
QA_Check_Time             3.598761
Total_Time              257.936179
Wait_time               219.625000
Name: mean, dtype: float64
Simulation 164 Complete
3 1 3 QC_Check_Time             7.025047
Testing_Machine_Time     27.741707
QA_Check_Time             3.598761
Total_Time              257.936179
Wait_time               219.625000
Name: mean, dtype: float64
Simulation 165 Complete
3 1 4 QC_Check_Time             7.025047
Testing_Machine_Time     27.741707
Q

...

Name: mean, dtype: float64
Simulation 550 Complete
7 8 2 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 551 Complete
7 8 3 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 552 Complete
7 8 4 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 553 Complete
7 8 5 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 554 Complete
7 8 6 QC_Check_Time            5.471183
T

8 3 8 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 593 Complete
8 3 9 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 594 Complete
8 4 1 QC_Check_Time            5.659019
Testing_Machine_Time    18.804967
QA_Check_Time            3.667348
Total_Time              29.400401
Wait_time                1.317073
Name: mean, dtype: float64
Simulation 595 Complete
8 4 2 QC_Check_Time            5.603664
Testing_Machine_Time    18.556312
QA_Check_Time            3.657091
Total_Time              28.739978
Wait_time                0.952381
Name: mean, dtype: float64
Simulation 596 Complete
8 4 3 QC_Check_Time            5.603664
Testing_Machine_Time    18.556312
QA_Check_Time     

8 7 9 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 630 Complete
8 8 1 QC_Check_Time            5.663506
Testing_Machine_Time    18.533237
QA_Check_Time            3.616237
Total_Time              28.666445
Wait_time                0.857143
Name: mean, dtype: float64
Simulation 631 Complete
8 8 2 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 632 Complete
8 8 3 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 633 Complete
8 8 4 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time     

9 3 3 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 669 Complete
9 3 4 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 670 Complete
9 3 5 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 671 Complete
9 3 6 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time            3.719678
Total_Time              42.313142
Wait_time               15.346154
Name: mean, dtype: float64
Simulation 672 Complete
9 3 7 QC_Check_Time            5.420423
Testing_Machine_Time    17.841268
QA_Check_Time     

9 7 7 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 709 Complete
9 7 8 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 710 Complete
9 7 9 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time            3.665732
Total_Time              28.030961
Wait_time                0.659091
Name: mean, dtype: float64
Simulation 711 Complete
9 8 1 QC_Check_Time            5.663506
Testing_Machine_Time    18.533237
QA_Check_Time            3.616237
Total_Time              28.666445
Wait_time                0.857143
Name: mean, dtype: float64
Simulation 712 Complete
9 8 2 QC_Check_Time            5.471183
Testing_Machine_Time    18.244762
QA_Check_Time     

Here we run 2 QC, 4 testing equipments, 1 QA for 20x to determine an average of the total time.

In [124]:
sim = 0
avg = []
random.seed(54)  # This helps reproducing the results
for i in range(20):
   
    
                
    df=[]
    time_df = []
        
    #create an environment and start the item arrival process
    env = simpy.Environment()
    env.process(Item_Arrival(env, arrival, 2, 4, 1, qc_time, testmac_time, qa_time_min, qa_time_max))
    
    #execute
    env.run(until= sim_time)
        
    #save times
    df = pd.DataFrame(time_df)
    df['Wait_time'] = round(df['Total_Time'] - df['QC_Check_Time'] - df['Testing_Machine_Time'] - df['QA_Check_Time'])
        
    avg.append(df.describe().loc['mean'])
     
    print("Simulation %s Complete" %sim)
    sim += 1

Simulation 0 Complete
Simulation 1 Complete
Simulation 2 Complete
Simulation 3 Complete
Simulation 4 Complete
Simulation 5 Complete
Simulation 6 Complete
Simulation 7 Complete
Simulation 8 Complete
Simulation 9 Complete
Simulation 10 Complete
Simulation 11 Complete
Simulation 12 Complete
Simulation 13 Complete
Simulation 14 Complete
Simulation 15 Complete
Simulation 16 Complete
Simulation 17 Complete
Simulation 18 Complete
Simulation 19 Complete


Finalize that 2 QC, 4 testing equipments, and 1 QA would keep total time in process at 29.400401 minutes and average wait time at 1.317073 minutes, which is the least combination of resources. 

In [125]:
# turn into dataframe for analysis
dfa = pd.DataFrame(avg)

print(dfa.shape)
dfa.head()

(20, 5)


,QC_Check_Time,Testing_Machine_Time,QA_Check_Time,Total_Time,Wait_time
mean,5.659019,18.804967,3.667348,29.400401,1.317073
mean,4.490678,15.768751,3.852073,25.272823,1.105263
mean,7.169407,19.508615,3.613740,32.670049,2.375000
mean,5.982338,15.592806,3.852379,30.220590,4.826087
mean,4.276114,19.588404,3.883469,28.986807,1.285714


In [131]:
print("Total Time:", dfa['Total_Time'].mean(), "\nWait Time:", dfa['Wait_time'].mean())


Total Time: 29.510186828251374 
Wait Time: 2.1031917437582113
